In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telco-resampled/Telco_resampled.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('ticks')
plt.rcParams["figure.figsize"] = (10,8)

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [3]:
df = pd.read_csv("/kaggle/input/telco-resampled/Telco_resampled.csv")

In [4]:
df.head()

,tenure,MonthlyCharges,TotalCharges,internet_cnt,Contract,InternetService_DSL,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,gender,SeniorCitizen,Partner,Dependents,PhoneService,PaperlessBilling,Churn
0,1,29.85,29.85,1,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,1,0,0,1,0
1,34,56.95,1889.50,1,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0,0,1,0,0
2,45,42.30,1840.75,1,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,0,0,0,0,0
3,22,89.10,1949.40,1,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1,0,0,1,1,1,0
4,10,29.75,301.90,1,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0,0,0,0


In [22]:
df.columns

Index(['tenure', 'MonthlyCharges', 'TotalCharges', 'internet_cnt', 'Contract',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService',
       'PaperlessBilling', 'Churn'],
      dtype='object')

## lightGBM

In [23]:
# import pandas as pd
# from sklearn.model_selection import train_test_split, GridSearchCV
# from lightgbm import LGBMClassifier
# from sklearn.metrics import accuracy_score

# data = df.copy()

# # 불필요한 컬럼 제거
# # data.drop(['Unnamed: 0', 'CustomerID'], axis=1, inplace=True)

# # 특성과 라벨 분리
# X = data.drop('Churn', axis=1)
# y = data['Churn']

# # 데이터 분할
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# params = {'verbose': -1}

# # 모델 설정
# model = LGBMClassifier()

# # 하이퍼파라미터 그리드 설정
# param_grid = {
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'max_depth': [3, 5, 7],
#     'num_leaves': [31, 62, 124],
# }

# # Grid Search 설정
# grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', verbose=1)

# # Grid Search 실행
# grid_search.fit(X_train, y_train)

# # 최적의 하이퍼파라미터 출력
# print("Best parameters:", grid_search.best_params_)
# print("Best accuracy: {:.4f}".format(grid_search.best_score_))

# # 테스트 데이터에 대한 성능 평가
# best_model = grid_search.best_estimator_
# predictions = best_model.predict(X_test)
# accuracy = accuracy_score(y_test, predictions)
# print("Test accuracy: {:.4f}".format(accuracy))


In [6]:
import lightgbm as lgb
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.model_selection import learning_curve
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMClassifier

data = df.copy()

# 특성과 라벨 분리
X = data.drop('Churn', axis=1)
y = data['Churn']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 설정
model = LGBMClassifier()

# Best parameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300, 'num_leaves': 62}


# 테스트 데이터에서 라벨이 1인 데이터 추출
mask = (y_test == 1)
class_1_data_X = X_test[mask]
class_1_data_y = y_test[mask]

# 라벨이 1인 데이터에서 1/3만 랜덤하게 추출
random_indices = np.random.choice(class_1_data_X.index, size=int(len(class_1_data_X)/3), replace=False)
reduced_class_1_data_X = class_1_data_X.loc[random_indices]
reduced_class_1_data_y = class_1_data_y.loc[random_indices]

# 라벨이 0인 데이터와 결합
class_0_data_X = X_test[y_test == 0]
class_0_data_y = y_test[y_test == 0]

# 최종 테스트 데이터셋 재구성
final_test_X = pd.concat([class_0_data_X, reduced_class_1_data_X])
final_test_y = pd.concat([class_0_data_y, reduced_class_1_data_y])

# 데이터셋 재구성 확인
print(final_test_X.shape, final_test_y.shape)

X_test = final_test_X.copy(deep=True)
y_test= final_test_y.copy(deep=True)


# 최적의 파라미터로 모델 재학습
# 모델 파라미터 설정
params = {
    'learning_rate': 0.1,
    'max_depth': 7,
    'n_estimators': 300,
    'num_leaves': 62
}

# LightGBM 모델 생성 및 학습
model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train)

# 성능 지표 계산
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')

# 전체 성능 지표 출력
# print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy: {accuracy:.4f}, AUC: {auc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

# 클래스 별 성능 지표 출력
print(classification_report(y_test, y_pred))

(1398, 18) (1398,)
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4161, number of negative: 4099
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001817 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 614
[LightGBM] [Info] Number of data points in the train set: 8260, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503753 -> initscore=0.015012
[LightGBM] [Info] Start training from score 0.015012
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [10]:
# print("Best parameters:", grid_search.best_params_)
# print("Best accuracy: {:.4f}".format(grid_search.best_score_))
# accuracy = accuracy_score(y_test, predictions)
# print("Test accuracy: {:.4f}".format(accuracy))

# Best parameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300, 'num_leaves': 62}
# Best accuracy: 0.8459
# Test accuracy: 0.8640

In [12]:
# 전체 성능 지표 출력
# print(f"Best Parameters: {grid_search.best_params_}")
print(f"Accuracy: {accuracy:.4f}, AUC: {auc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

# 클래스 별 성능 지표 출력
print(classification_report(y_test, y_pred))

# # 학습 곡선 출력
# plot_learning_curve(final_model, "Learning Curve (LGBM)", X_train, y_train, cv=5)
# plt.show()

Accuracy: 0.8326, AUC: 0.9266, Precision: 0.5943, Recall: 0.9431, F1-Score: 0.7292
              precision    recall  f1-score   support

           0       0.98      0.80      0.88      1064
           1       0.59      0.94      0.73       334

    accuracy                           0.83      1398
   macro avg       0.79      0.87      0.80      1398
weighted avg       0.89      0.83      0.84      1398



In [13]:
# LightGBM 모델 생성 및 학습
# model = lgb.LGBMClassifier(**params)
# model.fit(X_train, y_train)

# 피처 중요도 추출
feature_importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values(by='importance', ascending=False)

# 피처 중요도 출력
print(feature_importances)


                                    feature  importance
1                            MonthlyCharges        3973
2                              TotalCharges        3020
0                                    tenure        1975
12                                   gender         438
17                         PaperlessBilling         336
4                                  Contract         330
10           PaymentMethod_Electronic check         330
14                                  Partner         284
13                            SeniorCitizen         238
15                               Dependents         238
8   PaymentMethod_Bank transfer (automatic)         189
11               PaymentMethod_Mailed check         186
9     PaymentMethod_Credit card (automatic)         169
16                             PhoneService          65
3                              internet_cnt          64
6               InternetService_Fiber optic          51
5                       InternetService_DSL     

In [18]:
# 중요도가 가장 낮은 n개 피처 제외
n = 5  # 제외할 피처의 수
features_to_exclude = feature_importances.head(n)['feature']
X_train_reduced = X_train.drop(columns=features_to_exclude)
X_test_reduced = X_test.drop(columns=features_to_exclude)

# X_train_reduced.head()
X_test_reduced.head()

,internet_cnt,Contract,InternetService_DSL,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,SeniorCitizen,Partner,Dependents,PhoneService,PaperlessBilling
4139,0,0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0,0,0,1,0
1692,1,2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0,0,0,1,1
2692,1,2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1,0,0,1
321,1,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0,1,0,1,0
39,1,1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1,1,1,1,1


In [19]:
# 모델 재학습
model_reduced = lgb.LGBMClassifier(**params)
model_reduced.fit(X_train_reduced, y_train)
new_score = model_reduced.score(X_test_reduced, y_test)  # 성능 평가

print(f'Model accuracy with reduced features: {new_score}')

# 성능 지표 계산
y_pred = model_reduced.predict(X_test_reduced)
y_proba = model_reduced.predict_proba(X_test_reduced)[:, 1]
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')

# 피처 중요도 추출
feature_importances = pd.DataFrame({
    'feature': X_train_reduced.columns,
    'importance': model_reduced.feature_importances_
}).sort_values(by='importance', ascending=False)

# 피처 중요도 출력
print(feature_importances)

# 전체 성능 지표 출력
print(f"Accuracy: {accuracy:.4f}, AUC: {auc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

# 클래스 별 성능 지표 출력
print(classification_report(y_test, y_pred))

Model accuracy with reduced features: 0.7253218884120172
                                    feature  importance
13                         PaperlessBilling        2448
10                                  Partner        2143
11                               Dependents        1890
1                                  Contract        1791
9                             SeniorCitizen        1420
7            PaymentMethod_Electronic check        1174
0                              internet_cnt        1143
12                             PhoneService        1093
6     PaymentMethod_Credit card (automatic)        1091
2                       InternetService_DSL        1078
5   PaymentMethod_Bank transfer (automatic)        1048
8                PaymentMethod_Mailed check         928
3               InternetService_Fiber optic         644
4                        InternetService_No         324
Accuracy: 0.7253, AUC: 0.8295, Precision: 0.4599, Recall: 0.8593, F1-Score: 0.5992
              precis

### Tenure 변환 & Charge_Ratio 추가

In [22]:
from sklearn.preprocessing import FunctionTransformer


## 피처 엔지니어링 1
# 월별 대비 총 지출 비율
data['Charge_Ratio'] = data['TotalCharges'] / data['MonthlyCharges']

# tenure의 로그 변환
transformer = FunctionTransformer(np.log1p, validate=True)
data['tenure_transformed'] = transformer.transform(data[['tenure']])

# 학습 및 테스트 데이터 분할
X = data.drop(['Churn','tenure'], axis=1)
## Charge도 추가 드롭
X = data.drop(['Churn','tenure','TotalCharges','MonthlyCharges'], axis=1)
y = data['Churn']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#X데이터 1:3비율로 다시 imbalancing

# 테스트 데이터에서 라벨이 1인 데이터 추출
mask = (y_test == 1)
class_1_data_X = X_test[mask]
class_1_data_y = y_test[mask]

# 라벨이 1인 데이터에서 1/3만 랜덤하게 추출
random_indices = np.random.choice(class_1_data_X.index, size=int(len(class_1_data_X)/3), replace=False)
reduced_class_1_data_X = class_1_data_X.loc[random_indices]
reduced_class_1_data_y = class_1_data_y.loc[random_indices]

# 라벨이 0인 데이터와 결합
class_0_data_X = X_test[y_test == 0]
class_0_data_y = y_test[y_test == 0]

# 최종 테스트 데이터셋 재구성
final_test_X = pd.concat([class_0_data_X, reduced_class_1_data_X])
final_test_y = pd.concat([class_0_data_y, reduced_class_1_data_y])

# 데이터셋 재구성 확인
print(final_test_X.shape, final_test_y.shape)

X_test = final_test_X.copy(deep=True)
y_test= final_test_y.copy(deep=True)

# 데이터의 첫 5행 출력
# print(X_train.head())

# 모델 파라미터 설정
params = {
    'learning_rate': 0.1,
    'max_depth': 7,
    'n_estimators': 300,
    'num_leaves': 62,
    'verbose': -1
}

# LightGBM 모델 생성 및 학습
model = lgb.LGBMClassifier(**params)
model.fit(X_train, y_train)

# 성능 지표 계산
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')

# 피처 중요도 추출
feature_importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values(by='importance', ascending=False)

# 피처 중요도 출력
print(feature_importances)

# 전체 성능 지표 출력
print(f"Accuracy: {accuracy:.4f}, AUC: {auc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

# 클래스 별 성능 지표 출력
print(classification_report(y_test, y_pred))

(1398, 17) (1398,)
                                    feature  importance
15                             Charge_Ratio        5376
16                       tenure_transformed        1976
9                                    gender         761
7            PaymentMethod_Electronic check         516
11                                  Partner         504
14                         PaperlessBilling         496
1                                  Contract         430
12                               Dependents         421
10                            SeniorCitizen         373
5   PaymentMethod_Bank transfer (automatic)         341
0                              internet_cnt         286
6     PaymentMethod_Credit card (automatic)         286
8                PaymentMethod_Mailed check         271
3               InternetService_Fiber optic         266
13                             PhoneService         254
2                       InternetService_DSL         234
4                        Inte

In [15]:
# 피처 중요도 계산
feature_importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values(by='importance', ascending=True)

# 중요도가 가장 낮은 n개 피처 제외
n = 5  # 제외할 피처의 수
features_to_exclude = feature_importances.head(n)['feature']
X_train_reduced = X_train.drop(columns=features_to_exclude)
X_test_reduced = X_test.drop(columns=features_to_exclude)

X_train_reduced.head()

,MonthlyCharges,TotalCharges,Contract,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,gender,SeniorCitizen,Partner,Dependents,PaperlessBilling,Charge_Ratio,tenure_transformed
9031,96.80,2030.30,0,1.0,0.0,0.0,0.0,1,1,0,0,1,20.974174,3.091042
8100,20.20,20.20,0,0.0,0.0,0.0,1.0,1,0,0,0,0,1.000000,0.693147
8400,89.45,240.45,0,0.0,0.0,1.0,0.0,0,0,0,0,1,2.688094,1.386294
828,66.25,1533.80,1,0.0,0.0,0.0,1.0,0,0,1,1,1,23.151698,3.178054
6964,94.85,462.80,0,1.0,0.0,0.0,0.0,1,0,0,0,1,4.879283,1.791759


In [16]:
# 모델 재학습
model_reduced = lgb.LGBMClassifier(**params)
model_reduced.fit(X_train_reduced, y_train)
new_score = model_reduced.score(X_test_reduced, y_test)  # 성능 평가

print(f'Model accuracy with reduced features: {new_score}')

# 성능 지표 계산
y_pred = model_reduced.predict(X_test_reduced)
y_proba = model_reduced.predict_proba(X_test_reduced)[:, 1]
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')

# 피처 중요도 추출
feature_importances = pd.DataFrame({
    'feature': X_train_reduced.columns,
    'importance': model_reduced.feature_importances_
}).sort_values(by='importance', ascending=False)

# 피처 중요도 출력
print(feature_importances)

# 전체 성능 지표 출력
print(f"Accuracy: {accuracy:.4f}, AUC: {auc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

# 클래스 별 성능 지표 출력
print(classification_report(y_test, y_pred))

Model accuracy with reduced features: 0.8333333333333334
                                    feature  importance
0                            MonthlyCharges        3408
1                              TotalCharges        2582
12                             Charge_Ratio        2103
13                       tenure_transformed         885
7                                    gender         406
2                                  Contract         343
5            PaymentMethod_Electronic check         301
11                         PaperlessBilling         297
9                                   Partner         258
10                               Dependents         213
8                             SeniorCitizen         192
6                PaymentMethod_Mailed check         162
3   PaymentMethod_Bank transfer (automatic)         156
4     PaymentMethod_Credit card (automatic)         133
Accuracy: 0.8333, AUC: 0.9293, Precision: 0.5951, Recall: 0.9461, F1-Score: 0.7306
              precis

In [31]:
X_train_reduced.columns

Index(['MonthlyCharges', 'TotalCharges', 'Contract',
       'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PaperlessBilling',
       'Charge_Ratio', 'tenure_transformed'],
      dtype='object')

In [32]:
X_train_reduced.head()

,MonthlyCharges,TotalCharges,Contract,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,gender,SeniorCitizen,Partner,Dependents,PaperlessBilling,Charge_Ratio,tenure_transformed
9031,96.80,2030.30,0,1.0,0.0,0.0,0.0,1,1,0,0,1,20.974174,3.091042
8100,20.20,20.20,0,0.0,0.0,0.0,1.0,1,0,0,0,0,1.000000,0.693147
8400,89.45,240.45,0,0.0,0.0,1.0,0.0,0,0,0,0,1,2.688094,1.386294
828,66.25,1533.80,1,0.0,0.0,0.0,1.0,0,0,1,1,1,23.151698,3.178054
6964,94.85,462.80,0,1.0,0.0,0.0,0.0,1,0,0,0,1,4.879283,1.791759


## Random Forest Classifier

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler


data = df.copy()

# 데이터 준비
X = data.drop('Churn', axis=1)
y = data['Churn']

# 데이터를 학습 및 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123, stratify=y)

# 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 그리드 서치를 위한 하이퍼파라미터 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# 랜덤포레스트와 그리드서치 객체 생성
rf = RandomForestClassifier(random_state=123)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='f1', verbose=2)

# 그리드 서치 실행
grid_search.fit(X_train_scaled, y_train)

# 최적 파라미터와 그 때의 F1 스코어
best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_params, best_score


Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.2s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.4s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   3.5s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   1.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   1.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   1.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   2.1s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=300; tot

({'bootstrap': False,
  'max_depth': 20,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'n_estimators': 200},
 0.8946285928873221)

[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   1.3s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   2.8s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time=   4.0s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=300; total time=   4.1s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   2.5s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   4.4s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   1.3s
[CV] END bootstrap=False, max_depth=20, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   1.3s
[CV] END bootstrap=False, max_depth=20, min_samples_le

In [ ]:
# ({'bootstrap': False,
#   'max_depth': 20,
#   'min_samples_leaf': 1,
#   'min_samples_split': 2,
#   'n_estimators': 200},
#  0.8946285928873221)

In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# 최적의 하이퍼파라미터 설정
best_params = {
    'bootstrap': False,
    'max_depth': 20,
    'min_samples_leaf': 1,
    'min_samples_split': 2,
    'n_estimators': 200}

# 최적의 파라미터를 사용하여 RandomForest 모델 생성
model = RandomForestClassifier(**best_params, random_state=42)

# 표준화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 훈련
model.fit(X_train_scaled, y_train)

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test_scaled)

# 성능 평가
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("AUC Score:", roc_auc_score(y_test, y_pred))

# 피처 중요도 출력
print("Feature Importances:")
print(model.feature_importances_)

print('-'*100)

# Predictions and evaluation
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
feature_importances = model.feature_importances_

# Print results and best parameters
best_params = grid_search.best_params_

# 전체 성능 지표 출력
print(f"Accuracy: {accuracy:.4f}, AUC: {auc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

# 클래스 별 성능 지표 출력
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.85      0.90      1033
           1       0.87      0.96      0.91      1033

    accuracy                           0.91      2066
   macro avg       0.91      0.91      0.91      2066
weighted avg       0.91      0.91      0.91      2066

AUC Score: 0.9070667957405615
Feature Importances:
[0.19030402 0.20288622 0.21375364 0.01009161 0.12393029 0.01217731
 0.04333254 0.02062486 0.01013143 0.01113974 0.03430211 0.00990196
 0.02540533 0.01766168 0.02200901 0.01891874 0.00717724 0.02625227]
----------------------------------------------------------------------------------------------------
Accuracy: 0.9071, AUC: 0.9071, Precision: 0.8685, Recall: 0.9593, F1-Score: 0.9117
              precision    recall  f1-score   support

           0       0.95      0.85      0.90      1033
           1       0.87      0.96      0.91      1033

    accuracy                           0.91      

In [50]:
# 피처 중요도 추출
feature_importances = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model.feature_importances_
}).sort_values(by='importance', ascending=False)

# 피처 중요도 출력
print(feature_importances)

'''
                                    feature  importance
2                              TotalCharges    0.213754
1                            MonthlyCharges    0.202886
0                                    tenure    0.190304
4                                  Contract    0.123930
6               InternetService_Fiber optic    0.043333
10           PaymentMethod_Electronic check    0.034302
17                         PaperlessBilling    0.026252
12                                   gender    0.025405
14                                  Partner    0.022009
7                        InternetService_No    0.020625
15                               Dependents    0.018919
13                            SeniorCitizen    0.017662
5                       InternetService_DSL    0.012177
9     PaymentMethod_Credit card (automatic)    0.011140
8   PaymentMethod_Bank transfer (automatic)    0.010131
3                              internet_cnt    0.010092
11               PaymentMethod_Mailed check    0.009902
16                             PhoneService    0.007177
'''

                                    feature  importance
2                              TotalCharges    0.213754
1                            MonthlyCharges    0.202886
0                                    tenure    0.190304
4                                  Contract    0.123930
6               InternetService_Fiber optic    0.043333
10           PaymentMethod_Electronic check    0.034302
17                         PaperlessBilling    0.026252
12                                   gender    0.025405
14                                  Partner    0.022009
7                        InternetService_No    0.020625
15                               Dependents    0.018919
13                            SeniorCitizen    0.017662
5                       InternetService_DSL    0.012177
9     PaymentMethod_Credit card (automatic)    0.011140
8   PaymentMethod_Bank transfer (automatic)    0.010131
3                              internet_cnt    0.010092
11               PaymentMethod_Mailed check    0

'\n                                    feature  importance\n2                              TotalCharges    0.213754\n1                            MonthlyCharges    0.202886\n0                                    tenure    0.190304\n4                                  Contract    0.123930\n6               InternetService_Fiber optic    0.043333\n10           PaymentMethod_Electronic check    0.034302\n17                         PaperlessBilling    0.026252\n12                                   gender    0.025405\n14                                  Partner    0.022009\n7                        InternetService_No    0.020625\n15                               Dependents    0.018919\n13                            SeniorCitizen    0.017662\n5                       InternetService_DSL    0.012177\n9     PaymentMethod_Credit card (automatic)    0.011140\n8   PaymentMethod_Bank transfer (automatic)    0.010131\n3                              internet_cnt    0.010092\n11               PaymentMeth

In [51]:
best_params

{'bootstrap': False,
 'max_depth': 20,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}